<a href="https://www.kaggle.com/code/bharath4504/major-project?scriptVersionId=237257122" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
import pandas as pd
import os
import torch
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from sklearn.model_selection import train_test_split


In [ ]:
import warnings
warnings.filterwarnings("ignore")

<h1> Dataset Loading</h1>

In [ ]:
train_df = pd.read_csv("/kaggle/input/paddy-disease-classification/train.csv")

In [ ]:
train_df.head()

In [ ]:
train_df.shape

In [ ]:
# Define image transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Required for ViT
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])  # Normalize
])

In [ ]:
# Create a label mapping (convert class names to numbers)
label_mapping = {label: idx for idx, label in enumerate(train_df['label'].unique())}

In [ ]:
# Define Custom Dataset Class
class PaddyDataset(Dataset):
    def __init__(self, dataframe, image_dir, transform=None):
        self.dataframe = dataframe
        self.image_dir = image_dir
        self.transform = transform

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        img_name = self.dataframe.iloc[idx]["image_id"]  # Keep original filename
        class_name = self.dataframe.iloc[idx]["label"]   # Get the class label
        img_path = os.path.join(self.image_dir, class_name, img_name)  # Corrected path
    
        # Debug: Print file path
        if not os.path.exists(img_path):
            print(f"File not found: {img_path}")
    
        image = Image.open(img_path).convert("RGB")
        
        if self.transform:
            image = self.transform(image)
    
        label = label_mapping[class_name]  # Use label mapping for encoding
        return image, label


In [ ]:
# Define dataset paths
train_images_path = "/kaggle/input/paddy-disease-classification/train_images"

In [ ]:
# Train-validation-test split (70% train, 20% val, 10% test)
train_df, test_df = train_test_split(train_df, test_size=0.1, stratify=train_df['label'], random_state=42)
train_df, val_df = train_test_split(train_df, test_size=2/9, stratify=train_df['label'], random_state=42) 

In [ ]:
# Create dataset instances
train_dataset = PaddyDataset(train_df, train_images_path, transform=transform)
val_dataset = PaddyDataset(val_df, train_images_path, transform=transform)
test_dataset = PaddyDataset(test_df, train_images_path, transform=transform)

In [ ]:

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [ ]:
# Check if DataLoader is working
images, labels = next(iter(train_loader))
print(f"Batch Image Shape: {images.shape}, Labels: {labels[:10]}")


<h1>Vision Transformers</h1>

<h1>Model Building</h1>

In [ ]:
from transformers import ViTForImageClassification
import torch

# Load Pretrained Vision Transformer Model
model = ViTForImageClassification.from_pretrained(
    "google/vit-base-patch16-224",
    num_labels=10,  # 10 classes
    ignore_mismatched_sizes=True  # Fixes the size mismatch error
)

In [ ]:
# Move Model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


# Print Model Summary
print(model)

<h1>Training</h1>

In [ ]:
import torch
import torch.optim as optim
import torch.nn as nn
from torch.utils.data import DataLoader

# Define Hyperparameters
batch_size = 32
epochs = 5
learning_rate = 2e-5  # Standard for fine-tuning transformers

# Define Loss Function & Optimizer
criterion = nn.CrossEntropyLoss()  # For multi-class classification
optimizer = optim.AdamW(model.parameters(), lr=learning_rate)


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


In [ ]:
train_loss_history = []
val_loss_history = []
train_acc_history = []
val_acc_history = []

best_acc = 0.0  # Track best model accuracy

scaler = torch.cuda.amp.GradScaler()  # For mixed precision training

epochs = 10
for epoch in range(epochs):
    model.train()
    total_train_loss, correct_train, total_train = 0, 0, 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()

        # Mixed Precision Training
        with torch.cuda.amp.autocast():
            outputs = model(images)
            if hasattr(outputs, "logits"):  # Ensure compatibility with ViT
                outputs = outputs.logits
            loss = criterion(outputs, labels)

        # Backpropagation with AMP
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        total_train_loss += loss.item()
        _, preds = torch.max(outputs, 1)
        correct_train += (preds == labels).sum().item()
        total_train += labels.size(0)

    # Compute Train Loss & Accuracy
    train_loss = total_train_loss / len(train_loader)
    train_acc = correct_train / total_train
    train_loss_history.append(train_loss)
    train_acc_history.append(train_acc)

    # Validation Loop
    model.eval()
    total_val_loss, correct_val, total_val = 0, 0, 0

    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)

            with torch.cuda.amp.autocast():
                outputs = model(images)
                if hasattr(outputs, "logits"):
                    outputs = outputs.logits
                loss = criterion(outputs, labels)

            total_val_loss += loss.item()
            _, preds = torch.max(outputs, 1)
            correct_val += (preds == labels).sum().item()
            total_val += labels.size(0)

    # Compute Validation Loss & Accuracy
    val_loss = total_val_loss / len(val_loader)
    val_acc = correct_val / total_val
    val_loss_history.append(val_loss)
    val_acc_history.append(val_acc)

    # Save Best Model
    if val_acc > best_acc:
        best_acc = val_acc
        torch.save(model.state_dict(), "best_model.pth")

    print(f"Epoch {epoch+1}/{epochs} - Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}, Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}")

print("Training Complete ✅")


<h1> Plots</h1>

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Define tick intervals
x_ticks = np.arange(1, epochs + 1, 1)  # X-axis (Epochs) gap of 1
# y_loss_ticks = np.arange(0, max(cnn_train_losses + cnn_val_losses) + 0.1, 0.1)  # Y-axis (Loss) gap of 0.1
# y_acc_ticks = np.arange(0.7, 1.1, 0.1)  # Y-axis (Accuracy) gap of 0.1 (assuming accuracy is 0-1)

# Plot Training & Validation Accuracy
plt.figure(figsize=(8, 5))
plt.plot(range(1, epochs + 1), train_acc_history, marker='o', color='red', label='Train Accuracy')
plt.plot(range(1, epochs + 1), val_acc_history, marker='o', color='blue', label='Validation Accuracy')
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.title("ViT Training & Validation Accuracy Over Epochs")
# plt.xticks(x_ticks)  # Set X-axis ticks
# plt.yticks(y_acc_ticks)  # Set Y-axis ticks
plt.legend()
plt.grid(True)
plt.show()

# Plot Training & Validation Loss
plt.figure(figsize=(8, 5))
plt.plot(range(1, epochs + 1), train_loss_history, marker='o', color='red', label='Train Loss')
plt.plot(range(1, epochs + 1), val_loss_history, marker='o', color='blue', label='Validation Loss')
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.title("ViT Training & Validation Loss Over Epochs")
# plt.xticks(x_ticks)  # Set X-axis ticks
# plt.yticks(y_loss_ticks)  # Set Y-axis ticks
plt.legend()
plt.grid(True)
plt.show()



<h1>Evaluation</h1>

In [ ]:
# Set the model to evaluation mode
model.eval()

# Lists to store test loss and accuracy
test_losses = []
test_accuracies = []

criterion = nn.CrossEntropyLoss()

# List to store predictions
predictions = []

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)

        # Forward pass
        outputs = model(images)

        # Extract logits if using a Vision Transformer
        if hasattr(outputs, "logits"):
            outputs = outputs.logits

        loss = criterion(outputs, labels)
        test_losses.append(loss.item())  # Store batch loss

        _, predicted = torch.max(outputs, 1)
        predictions.extend(predicted.cpu().numpy())

        # Calculate accuracy for this batch
        batch_accuracy = (predicted == labels).sum().item() / labels.size(0)
        test_accuracies.append(batch_accuracy)  # Store batch accuracy

# Compute overall Test Loss & Accuracy
avg_test_loss = sum(test_losses) / len(test_losses)
avg_test_accuracy = sum(test_accuracies) / len(test_accuracies)



In [ ]:
print(f"Test Loss: {avg_test_loss:.4f}\nTest Accuracy: {avg_test_accuracy:.4f}")

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# Collect true labels from the test loader
true_labels = []
for images, labels in test_loader:
    true_labels.extend(labels.numpy())

# Generate Classification Report
report = classification_report(true_labels, predictions)
print("Classification Report:\n")
print(report)

# Generate Confusion Matrix
cm = confusion_matrix(true_labels, predictions)

# Plotting the Confusion Matrix
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=np.unique(true_labels), 
            yticklabels=np.unique(true_labels))
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Confusion Matrix')
plt.show()


<h1>CNN</h1>

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class CNNModel(nn.Module):
    def __init__(self, num_classes):
        super(CNNModel, self).__init__()
        
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=1)
        
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        
        self.fc1 = nn.Linear(128 * 28 * 28, 512)  # Assuming input image size is 224x224
        self.fc2 = nn.Linear(512, num_classes)
        
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        
        x = torch.flatten(x, start_dim=1)
        
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        
        return x

# Instantiate the CNN model
num_classes = len(label_mapping)  # Ensure this matches the dataset
cnn_model = CNNModel(num_classes)

print(cnn_model)


In [ ]:
import torch
import torch.optim as optim
import torch.nn as nn
from torch.cuda.amp import autocast, GradScaler  # For Mixed Precision Training

# Define device
cnn_device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Move model to device
cnn_model = cnn_model.to(cnn_device)

# Define Loss Function and Optimizer
cnn_criterion = nn.CrossEntropyLoss()
cnn_optimizer = optim.AdamW(cnn_model.parameters(), lr=0.001, weight_decay=1e-4)  # AdamW is more stable
cnn_scaler = GradScaler()  # For mixed precision

# Training Loop
cnn_num_epochs = 10  
cnn_train_losses = []
cnn_train_accuracies = []
cnn_val_losses = []
cnn_val_accuracies = []

for cnn_epoch in range(cnn_num_epochs):
    cnn_model.train()
    cnn_running_loss = 0.0
    cnn_correct_train = 0
    cnn_total_train = 0

    for cnn_images, cnn_labels in train_loader:
        cnn_images, cnn_labels = cnn_images.to(cnn_device, non_blocking=True), cnn_labels.to(cnn_device, non_blocking=True)

        cnn_optimizer.zero_grad()

        # Mixed precision training
        with autocast():
            cnn_outputs = cnn_model(cnn_images)
            cnn_loss = cnn_criterion(cnn_outputs, cnn_labels)

        cnn_scaler.scale(cnn_loss).backward()
        cnn_scaler.step(cnn_optimizer)
        cnn_scaler.update()

        cnn_running_loss += cnn_loss.item()

        _, cnn_predicted_train = torch.max(cnn_outputs, 1)
        cnn_correct_train += (cnn_predicted_train == cnn_labels).sum().item()
        cnn_total_train += cnn_labels.size(0)

    cnn_avg_train_loss = cnn_running_loss / len(train_loader)
    cnn_train_losses.append(cnn_avg_train_loss)
    cnn_train_accuracy = cnn_correct_train / cnn_total_train
    cnn_train_accuracies.append(cnn_train_accuracy)

    # Validation Phase
    cnn_model.eval()
    cnn_val_loss = 0.0
    cnn_correct_val = 0
    cnn_total_val = 0

    with torch.no_grad():
        for cnn_images, cnn_labels in val_loader:
            cnn_images, cnn_labels = cnn_images.to(cnn_device, non_blocking=True), cnn_labels.to(cnn_device, non_blocking=True)

            with autocast():  # Use mixed precision in validation too
                cnn_outputs = cnn_model(cnn_images)
                cnn_loss = cnn_criterion(cnn_outputs, cnn_labels)

            cnn_val_loss += cnn_loss.item()

            _, cnn_predicted_val = torch.max(cnn_outputs, 1)
            cnn_correct_val += (cnn_predicted_val == cnn_labels).sum().item()
            cnn_total_val += cnn_labels.size(0)

    cnn_avg_val_loss = cnn_val_loss / len(val_loader)
    cnn_val_losses.append(cnn_avg_val_loss)
    cnn_val_accuracy = cnn_correct_val / cnn_total_val
    cnn_val_accuracies.append(cnn_val_accuracy)

    print(f"Epoch [{cnn_epoch+1}/{cnn_num_epochs}]: " 
          f" Train Accuracy: {cnn_train_accuracy:.4f}, "
          f" Train Loss: {cnn_avg_train_loss:.4f}, "
          f" Val Accuracy: {cnn_val_accuracy:.4f}, "
          f" Val Loss: {cnn_avg_val_loss:.4f} ")


In [ ]:
import numpy as np

# Define tick intervals
x_ticks = np.arange(1, cnn_num_epochs + 1, 1)  # X-axis (Epochs) gap of 1
# y_loss_ticks = np.arange(0, max(cnn_train_losses + cnn_val_losses) + 0.1, 0.1)  # Y-axis (Loss) gap of 0.1
# y_acc_ticks = np.arange(0.7, 1.1, 0.1)  # Y-axis (Accuracy) gap of 0.1 (assuming accuracy is 0-1)

# Plot Training & Validation Accuracy
plt.figure(figsize=(8, 5))
plt.plot(range(1, cnn_num_epochs + 1), cnn_train_accuracies, marker='o', color='red', label='Train Accuracy')
plt.plot(range(1, cnn_num_epochs + 1), cnn_val_accuracies, marker='o', color='blue', label='Validation Accuracy')
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.title("CNN Training & Validation Accuracy Over Epochs")
# plt.xticks(x_ticks)  # Set X-axis ticks
# plt.yticks(y_acc_ticks)  # Set Y-axis ticks
plt.legend()
plt.grid(True)
plt.show()

# Plot Training & Validation Loss
plt.figure(figsize=(8, 5))
plt.plot(range(1, cnn_num_epochs + 1), cnn_train_losses, marker='o', color='red', label='Train Loss')
plt.plot(range(1, cnn_num_epochs + 1), cnn_val_losses, marker='o', color='blue', label='Validation Loss')
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.title("CNN Training & Validation Loss Over Epochs")
# plt.xticks(x_ticks)  # Set X-axis ticks
# plt.yticks(y_loss_ticks)  # Set Y-axis ticks
plt.legend()
plt.grid(True)
plt.show()



In [ ]:
# Set model to evaluation mode
cnn_model.eval()

# Initialize test loss and accuracy tracking
cnn_total_test_loss = 0
cnn_correct_test = 0
cnn_total_test = 0

cnn_test_losses = []
cnn_test_accuracies = []

criterion = nn.CrossEntropyLoss()  # Ensure it matches the training criterion

with torch.no_grad():
    for cnn_images, cnn_labels in test_loader:
        cnn_images, cnn_labels = cnn_images.to(cnn_device, non_blocking=True), cnn_labels.to(cnn_device, non_blocking=True)

        # Forward pass
        with autocast():  # Use mixed precision for efficiency
            cnn_outputs = cnn_model(cnn_images)
            cnn_loss = criterion(cnn_outputs, cnn_labels)

        # Accumulate loss and correct predictions
        cnn_total_test_loss += cnn_loss.item()
        _, cnn_predicted_test = torch.max(cnn_outputs, 1)

        cnn_correct_test += (cnn_predicted_test == cnn_labels).sum().item()
        cnn_total_test += cnn_labels.size(0)

# Compute Test Loss & Accuracy
cnn_avg_test_loss = cnn_total_test_loss / len(test_loader)
cnn_test_losses.append(cnn_avg_test_loss)

cnn_test_accuracy = cnn_correct_test / cnn_total_test
cnn_test_accuracies.append(cnn_test_accuracy)

print(f"Test Loss: {cnn_avg_test_loss:.4f}, Test Accuracy: {cnn_test_accuracy:.4f}")


In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# Initialize lists to store all true labels and predictions
all_true_labels = []
all_predictions = []

# Iterate over the test set and collect all true labels and predictions
with torch.no_grad():
    for cnn_images, cnn_labels in test_loader:
        cnn_images, cnn_labels = cnn_images.to(cnn_device, non_blocking=True), cnn_labels.to(cnn_device, non_blocking=True)

        # Forward pass
        with autocast():  # Use mixed precision for efficiency
            cnn_outputs = cnn_model(cnn_images)
            _, cnn_predicted = torch.max(cnn_outputs, 1)

        # Collect true labels and predictions
        all_true_labels.extend(cnn_labels.cpu().numpy())
        all_predictions.extend(cnn_predicted.cpu().numpy())

# Now, generate the Classification Report
CNNreport = classification_report(all_true_labels, all_predictions)
print("Classification Report:\n")
print(CNNreport)

# Generate Confusion Matrix
CNNcm = confusion_matrix(all_true_labels, all_predictions)

# Plotting the Confusion Matrix
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=np.unique(all_true_labels), 
            yticklabels=np.unique(all_true_labels))
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Confusion Matrix')
plt.show()


<h1> Comparitive Analysis</h1>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Plot Training & Validation Accuracy Comparison
plt.figure(figsize=(8, 5))
plt.plot(range(1, cnn_num_epochs + 1), cnn_train_accuracies, marker='o', color='red', linestyle='-', label='CNN Train Acc')
#plt.plot(range(1, cnn_num_epochs + 1), cnn_val_accuracies, marker='o', color='blue', linestyle='--', label='CNN Val Acc')
plt.plot(range(1, len(train_acc_history) + 1), train_acc_history, marker='s', color='orange', linestyle='-', label='ViT Train Acc')
#plt.plot(range(1, len(val_acc_history) + 1), val_acc_history, marker='s', color='green', linestyle='--', label='ViT Val Acc')
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.title("ViT vs CNN Training Accuracy Comparison")
plt.legend()
plt.grid(True)
plt.show()

plt.figure(figsize=(8, 5))
#plt.plot(range(1, cnn_num_epochs + 1), cnn_train_accuracies, marker='o', color='red', linestyle='-', label='CNN Train Acc')
plt.plot(range(1, cnn_num_epochs + 1), cnn_val_accuracies, marker='o', color='blue', linestyle='--', label='CNN Val Acc')
#plt.plot(range(1, len(train_acc_history) + 1), train_acc_history, marker='s', color='orange', linestyle='-', label='ViT Train Acc')
plt.plot(range(1, len(val_acc_history) + 1), val_acc_history, marker='s', color='green', linestyle='--', label='ViT Val Acc')
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.title("ViT vs CNN Validation Accuracy Comparison")
plt.legend()
plt.grid(True)
plt.show()


# Plot Training & Validation Loss Comparison
plt.figure(figsize=(8, 5))
plt.plot(range(1, cnn_num_epochs + 1), cnn_train_losses, marker='o', color='red', linestyle='-', label='CNN Train Loss')
#plt.plot(range(1, cnn_num_epochs + 1), cnn_val_losses, marker='o', color='blue', linestyle='--', label='CNN Val Loss')
plt.plot(range(1, len(train_loss_history) + 1), train_loss_history, marker='s', color='orange', linestyle='-', label='ViT Train Loss')
#plt.plot(range(1, len(val_loss_history) + 1), val_loss_history, marker='s', color='green', linestyle='--', label='ViT Val Loss')
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.title("ViT vs CNN Training Loss Comparison")
plt.legend()
plt.grid(True)
plt.show()

plt.figure(figsize=(8, 5))
#plt.plot(range(1, cnn_num_epochs + 1), cnn_train_losses, marker='o', color='red', linestyle='-', label='CNN Train Loss')
plt.plot(range(1, cnn_num_epochs + 1), cnn_val_losses, marker='o', color='blue', linestyle='--', label='CNN Val Loss')
#plt.plot(range(1, len(train_loss_history) + 1), train_loss_history, marker='s', color='orange', linestyle='-', label='ViT Train Loss')
plt.plot(range(1, len(val_loss_history) + 1), val_loss_history, marker='s', color='green', linestyle='--', label='ViT Val Loss')
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.title("ViT vs CNN Validation Loss Comparison")
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Test Metrics for CNN & ViT
test_metrics = ["Test Loss", "Test Accuracy"]
cnn_test_values = [cnn_avg_test_loss, cnn_test_accuracy]
vit_test_values = [avg_test_loss, avg_test_accuracy]

# X-axis positions
x = np.arange(len(test_metrics))

# Bar Width
width = 0.35  

# Plot Test Loss & Accuracy Comparison
plt.figure(figsize=(14, 8))
plt.bar(x - width/2, cnn_test_values, width, color='red', label="CNN")
plt.bar(x + width/2, vit_test_values, width, color='blue', label="ViT")

# Labels & Titles
plt.xlabel("Metrics")
plt.ylabel("Values")
plt.title("ViT vs CNN Test Performance Comparison")
plt.xticks(x, test_metrics)
plt.ylim(0, 1)  # Normalize scale for comparison
plt.legend()
# plt.grid(axis='y', linestyle='--', alpha=0.7)

# Display values on top of bars
for i, v in enumerate(cnn_test_values):
    plt.text(i - width/2, v + 0.02, f"{v:.4f}", ha='center', fontsize=12)
for i, v in enumerate(vit_test_values):
    plt.text(i + width/2, v + 0.02, f"{v:.4f}", ha='center', fontsize=12)

plt.show()


In [ ]:
plt.figure(figsize=(8, 7))
plt.bar(["CNN"], [cnn_test_accuracy], color='red', width=0.4, label="CNN Test Accuracy")
plt.bar(["ViT"], [avg_test_accuracy], color='blue', width=0.4, label="ViT Test Accuracy")

# Labels & Titles
plt.ylabel("Accuracy")
plt.title("CNN vs ViT Test Accuracy Comparison")
plt.ylim(0, 1)  # Normalize scale
plt.legend()
# plt.grid(axis='y', linestyle='--', alpha=0.7)

# Display values on bars
plt.text("CNN", cnn_test_accuracy + 0.02, f"{cnn_test_accuracy:.4f}", ha='center', fontsize=12, color="red")
plt.text("ViT", avg_test_accuracy + 0.02, f"{avg_test_accuracy:.4f}", ha='center', fontsize=12, color="blue")

plt.show()


In [ ]:
plt.figure(figsize=(8, 7))
plt.bar(["CNN"], [cnn_avg_test_loss], color='red', width=0.4, label="CNN Test Loss")
plt.bar(["ViT"], [avg_test_loss], color='blue', width=0.4, label="ViT Test Loss")

# Labels & Titles
plt.ylabel("Loss")
plt.title("CNN vs ViT Test Loss Comparison")
plt.ylim(0, 1.3)  # Normalize scale
plt.legend()
# plt.grid(axis='y', linestyle='--', alpha=0.7)

# Display values on bars
plt.text("CNN", cnn_avg_test_loss + 0.02, f"{cnn_avg_test_loss:.4f}", ha='center', fontsize=12, color="red")
plt.text("ViT", avg_test_loss + 0.02, f"{avg_test_loss:.4f}", ha='center', fontsize=12, color="blue")

plt.show()
